In [88]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from datetime import datetime

In [89]:
main_df = pd.read_csv("Resources/netflix_titles.csv")
main_df = main_df.dropna()
main_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
5,s6,TV Show,46,Serdar Akar,"Erdal Beşikçioğlu, Yasemin Allen, Melis Birkan...",Turkey,"July 1, 2017",2016,TV-MA,1 Season,"International TV Shows, TV Dramas, TV Mysteries",A genetics professor experiments with a treatm...


In [90]:
cleaned_df = main_df.rename(columns={'listed_in':'genre', 'cast':'actor'})

cleaned_df.actor = cleaned_df.actor.str.split(', ')
cleaned_df = cleaned_df.explode('actor').reset_index(drop=True)

cleaned_df.country = cleaned_df.country.str.split(', ')
cleaned_df = cleaned_df.explode('country').reset_index(drop=True)

cleaned_df.genre = cleaned_df.genre.str.split(', ')
cleaned_df = cleaned_df.explode('genre').reset_index(drop=True)

cleaned_df

,show_id,type,title,director,actor,country,date_added,release_year,rating,duration,genre,description
0,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
1,s2,Movie,7:19,Jorge Michel Grau,Demián Bichir,Mexico,"December 23, 2016",2016,TV-MA,93 min,International Movies,After a devastating earthquake hits Mexico Cit...
2,s2,Movie,7:19,Jorge Michel Grau,Héctor Bonilla,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
3,s2,Movie,7:19,Jorge Michel Grau,Héctor Bonilla,Mexico,"December 23, 2016",2016,TV-MA,93 min,International Movies,After a devastating earthquake hits Mexico Cit...
4,s2,Movie,7:19,Jorge Michel Grau,Oscar Serrano,Mexico,"December 23, 2016",2016,TV-MA,93 min,Dramas,After a devastating earthquake hits Mexico Cit...
...,...,...,...,...,...,...,...,...,...,...,...,...
113131,s7784,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,TV-14,111 min,International Movies,A scrappy but poor boy worms his way into a ty...
113132,s7784,Movie,Zubaan,Mozez Singh,Anita Shabdish,India,"March 2, 2019",2015,TV-14,111 min,Music & Musicals,A scrappy but poor boy worms his way into a ty...
113133,s7784,Movie,Zubaan,Mozez Singh,Chittaranjan Tripathy,India,"March 2, 2019",2015,TV-14,111 min,Dramas,A scrappy but poor boy worms his way into a ty...
113134,s7784,Movie,Zubaan,Mozez Singh,Chittaranjan Tripathy,India,"March 2, 2019",2015,TV-14,111 min,International Movies,A scrappy but poor boy worms his way into a ty...


In [91]:
# Updated ERD
# Show
# -
# show_id VARCHAR(255) PK
# title VARCHAR(255)
# rating VARCHAR(255)
# description VARCHAR(255)
# director VARCHAR(255)
# type VARCHAR(255)
# date_added VARCHAR(255)
# year INT


# Actors
# -
# actor_id INT PK
# actor VARCHAR(255)


# Country
# -
# country_id INT PK
# country VARCHAR(255)


# Genre
# -
# genre_id INT PK
# genre VARCHAR(255)


# Relations
# -
# show_id VARCHAR(255) FK - Show.show_id
# actor_id INT FK - Actors.actor_id
# country_id INT FK - Country.country_id
# genre_id INT FK - Genre.genre_id

In [92]:
# Create table for shows
show_df = cleaned_df[["show_id", "title", "director", "rating", "type", "date_added", "release_year", "description", ]]
show_df.to_csv('Resources/shows.csv', index=False)

show_df["date_added"] = pd.to_datetime(show_df["date_added"])

# remove year

show_df

<ipython-input-92-f63f5fb026a0>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  show_df["date_added"] = pd.to_datetime(show_df["date_added"])


,show_id,title,director,rating,type,date_added,release_year,description
0,s2,7:19,Jorge Michel Grau,TV-MA,Movie,2016-12-23,2016,After a devastating earthquake hits Mexico Cit...
1,s2,7:19,Jorge Michel Grau,TV-MA,Movie,2016-12-23,2016,After a devastating earthquake hits Mexico Cit...
2,s2,7:19,Jorge Michel Grau,TV-MA,Movie,2016-12-23,2016,After a devastating earthquake hits Mexico Cit...
3,s2,7:19,Jorge Michel Grau,TV-MA,Movie,2016-12-23,2016,After a devastating earthquake hits Mexico Cit...
4,s2,7:19,Jorge Michel Grau,TV-MA,Movie,2016-12-23,2016,After a devastating earthquake hits Mexico Cit...
...,...,...,...,...,...,...,...,...
113131,s7784,Zubaan,Mozez Singh,TV-14,Movie,2019-03-02,2015,A scrappy but poor boy worms his way into a ty...
113132,s7784,Zubaan,Mozez Singh,TV-14,Movie,2019-03-02,2015,A scrappy but poor boy worms his way into a ty...
113133,s7784,Zubaan,Mozez Singh,TV-14,Movie,2019-03-02,2015,A scrappy but poor boy worms his way into a ty...
113134,s7784,Zubaan,Mozez Singh,TV-14,Movie,2019-03-02,2015,A scrappy but poor boy worms his way into a ty...


In [93]:
show_df['title'].value_counts()

Nymphomaniac: Volume II                               216
The Congress                                          210
Watership Down                                        189
The Danish Girl                                       180
Gosford Park                                          180
                                                     ... 
Jen Kirkman: I'm Gonna Die Alone (And I Feel Fine)      1
Chris Rock: Tamborine                                   1
Dream Big: Engineering Our World                        1
Edmilson Filho: Notas, Comedy about Relationships       1
John Mulaney: Kid Gorgeous at Radio City                1
Name: title, Length: 4808, dtype: int64

In [94]:
# Create table for actors and transform
actor_df = cleaned_df[["actor"]]

actor_df = actor_df.drop_duplicates()

actor_df.to_csv('Resources/actors.csv', index=False)
actor_df

,actor
0,Demián Bichir
2,Héctor Bonilla
4,Oscar Serrano
6,Azalia Ortiz
8,Octavio Michel
...,...
113118,Raaghav Chanana
113124,Meghna Malik
113127,Malkeet Rauni
113130,Anita Shabdish


In [95]:
# Create table for countries and transform
country_df = cleaned_df[["country"]]

country_df = country_df.drop_duplicates()

country_df.to_csv('Resources/countries.csv', index=False)
country_df

,country
0,Mexico
12,Singapore
30,United States
69,Turkey
99,Egypt
...,...
97863,Somalia
97865,Sudan
98627,Panama
110854,East Germany


In [96]:
country_df.value_counts()

country      
Zimbabwe         1
Liechtenstein    1
Finland          1
France           1
Georgia          1
                ..
Poland           1
Poland,          1
Portugal         1
Qatar            1
Afghanistan      1
Length: 105, dtype: int64

In [97]:
# Create table for genres and transform
genre_df = cleaned_df[["genre"]]

genre_df = genre_df.drop_duplicates()

genre_df.to_csv('Resources/genres.csv', index=False)
genre_df

,genre
0,Dramas
1,International Movies
12,Horror Movies
30,Action & Adventure
31,Independent Movies
32,Sci-Fi & Fantasy
69,International TV Shows
70,TV Dramas
71,TV Mysteries
140,Thrillers


In [64]:

# cleaned_df.actors = cleaned_df.actors.str.split(', ')
# cleaned_df.genre = cleaned_df.genre.str.split(', ')
# cleaned_df.country = cleaned_df.country.str.split(', ')

# cleaned_df = cleaned_df.explode('actors').reset_index(drop=True)
# cleaned_df = cleaned_df.explode('genre').reset_index(drop=True)
# cleaned_df = cleaned_df.explode('country').reset_index(drop=True)

# cleaned_df

In [18]:
# # Looking at values within columns

# unique_combo = main_df["listed_in"].unique()
# genre_list = []
# for unique in unique_combo:
#     genre_split = unique.split(', ')
#     for genre in genre_split:
#         if genre not in genre_list:
#             genre_list.append(gen|re)
# len(genre_list)

In [19]:
# movie_titles
# -
# ID VARCHAR PK
# show_name VARCHAR
# rating_ID INTEGER FK >- rating.rating_ID

# rating
# -
# rating_ID INTEGER PK
# rating_description VARCHAR

# listed_in
# -
# listed_in_ID INTEGER
# listed_in_description VARCHAR
# many to many

# info
# -
# ID VARHCAR FK >- movie_titles.ID
# listed_in_ID INTEGER FK >- listed_in.listed_in_ID
# type_ID VARCHAR FK >- type.type_ID
# date_added DATE
# rel_yr VARCHAR
# duration VARCHAR
# description VARCHAR

# country VARHCAR

# type
# -
# type_ID INTEGER PK
# type_desc VARCHAR
# not necessary

# cast
# -
# cast_ID INTEGER PK FK >- crew.cast_ID
# cast VARHCAR
# many to many

# director
# -
# director_ID INTEGER PK
# director VARCHAR
# not necessary

# crew
# -
# ID VARCHAR FK >- movie_titles.ID
# cast_ID INTGER FK
# director_ID INTEGER FK >- director.director_ID
# not necessary
